In [12]:
import pandas as pd
import numpy as np
import os, sys, warnings, io
from tqdm import tqdm_notebook as tqdm
import re, string
import nltk
from gensim.models import FastText
from gensim.test.utils import datapath
from gensim.models.keyedvectors import Word2VecKeyedVectors

warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.dirname('../../modules/')))

In [2]:
path_prepared = '../../dataset/prepared/'
train = np.load(path_prepared+'train.npy')
test = np.load(path_prepared+'test.npy')
validate = np.load(path_prepared+'validation.npy')

In [39]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

# fast_text = load_vectors('../../dataset/fast_text/wiki-news-300d-1M-subword')

In [27]:
fast_text_path = datapath('c:/gpr-19spring/dataset/fast_text/wiki-news-300d-1M-subword.vec')
model = Word2VecKeyedVectors.load_word2vec_format(fast_text_path)

In [28]:
type(model)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [56]:
# help(model)

In [34]:
train[0][0][2]

'played'

In [51]:
model.word_vec(train[0][0][2]).shape

(300,)

In [53]:
%%time
not_in_word_vec = []
for docs in (train, test, validate):
    for doc in tqdm(docs):
        for sent in doc:
            for word in sent:
                try : model.word_vec(word)
                except KeyError : not_in_word_vec.append(word)


Wall time: 504 ms


In [55]:
len(not_in_word_vec)

17095

In [49]:
%%time
def word2vec(doc):
    ret = []
    for sent in doc:
        ret_sent = []
        for word in sent:
            ret_sent.append(model.word_vec(word))
        ret.append(np.array(ret_sent))
    return np.array(ret)
save_file_name = ['train', 'test', 'validate']
save_path = '../../dataset/prepared/'
for i,docs in tqdm(enumerate((train, test, validate))):
    docs = np.array(list(map(word2vec, docs)))
    np.save(save_path+save_file_name[i]+'_fast_text', docs)

KeyError: "word 'casidy' not in vocabulary"

In [42]:
elmo = pd.read_hdf('../../dataset/elmo/elmo.hdf5')

ValueError: No dataset in HDF5 file.

In [44]:
import h5py
elmo = h5py.File('../../dataset/elmo/elmo.hdf5', 'r')

AttributeError: type object 'h5py.h5r.Reference' has no attribute '__reduce_cython__'